In [ ]:
from pathlib import Path
import joblib
import numpy as np
import pandas as pd

data_dir = Path("/data/processed")

X_full = joblib.load(data_dir / "X_full.joblib")
y_full = joblib.load(data_dir / "y_full.joblib")

print("Type X_full :", type(X_full))
print("Shape X_full:", X_full.shape)
print("Shape y_full:", np.array(y_full).shape)

if isinstance(X_full, pd.DataFrame):
    feature_names = X_full.columns
else:
    feature_names = pd.read_csv(
        "D:/Dataset/features.txt",
        delim_whitespace=True, header=None, names=["idx", "name"]
    )["name"].values
    X_full = pd.DataFrame(X_full, columns=feature_names)

activity_labels_df = pd.read_csv(
    "D:/Dataset/activity_labels.txt",
    sep=r"\s+", header=None, names=["id", "label"]
)
activity_labels = dict(zip(activity_labels_df["id"], activity_labels_df["label"]))

print("Colonnes X_full (extrait) :", feature_names[:5])


In [ ]:
from joblib import load

scaler_path = r"C:\Users\hp\Downloads\MLFLOW_HumanActivity\models\scaler.joblib"
model_path  = r"C:\Users\hp\Downloads\MLFLOW_HumanActivity\models\best_model.joblib"

scaler = load(scaler_path)
best_model = load(model_path)

print("Scaler et modèle chargés.")
print("Type du modèle :", type(best_model))


In [ ]:
X = X_full.values
y = np.array(y_full)

X_scaled = scaler.transform(X)

print("X_scaled shape :", X_scaled.shape)

n_background = min(2000, X_scaled.shape[0])
X_background = X_scaled[:n_background]

n_test_explain = min(500, X_scaled.shape[0])
X_explain = X_scaled[:n_test_explain]
y_explain = y[:n_test_explain]

print("Background pour SHAP :", X_background.shape)
print("Échantillon expliqué :", X_explain.shape)


In [ ]:
import shap
import matplotlib.pyplot as plt

shap.initjs()

print("Type du modèle :", type(best_model))


print("Utilisation directe de KernelExplainer (modèle SVC)")


background_small = shap.sample(X_background, 30, random_state=42)


X_explain_small = X_explain[:10]
y_explain_small = y_explain[:10]

explainer = shap.KernelExplainer(
    best_model.predict_proba,
    background_small
)

print(" Explainer créé, calcul des SHAP values...")
shap_values = explainer.shap_values(X_explain_small, nsamples=100)

print(" SHAP values calculées. Shapes :")
if isinstance(shap_values, list):
    print("Nombre de classes :", len(shap_values))
    print("Shape d'une matrice SHAP :", shap_values[0].shape)
else:
    print("Shape shap_values :", shap_values.shape)

shap.summary_plot(shap_values, X_explain_small, feature_names=feature_names)
shap.summary_plot(shap_values, X_explain_small, feature_names=feature_names, plot_type="bar")


In [ ]:
 import shap
import numpy as np


i = 3  


true_label = int(y_explain_small[i])
print("Vraie classe   :", true_label, "-", activity_labels.get(true_label, "Unknown"))

proba = best_model.predict_proba(X_explain_small[i].reshape(1, -1))[0]
class_ids = best_model.classes_         
pred_class_idx = int(np.argmax(proba))     
pred_class_label = int(class_ids[pred_class_idx])

print("Classe prédite :", pred_class_label, "-", activity_labels.get(pred_class_label, "Unknown"))
print("Probas prédictes :", dict(zip(class_ids, np.round(proba, 3))))


if isinstance(shap_values, list):

    sv_i = shap_values[pred_class_idx][i]         
    base_value = explainer.expected_value[pred_class_idx]
else:
   
    sv_i = shap_values[i]                          

    if sv_i.ndim == 2:
       
        sv_i = sv_i[:, pred_class_idx]

    base_value = explainer.expected_value

    if np.ndim(base_value) > 0:
        base_value = base_value[pred_class_idx]


print("Shape des SHAP values pour l'instance :", sv_i.shape)


expl = shap.Explanation(
    values=sv_i,                          
    base_values=base_value,
    data=X_explain_small[i],
    feature_names=feature_names
)

shap.plots.waterfall(expl, max_display=15)


In [ ]:
import numpy as np
import pandas as pd


print("X_explain_small shape :", X_explain_small.shape)
print("len(y_explain_small)  :", len(y_explain_small))

class_ids = best_model.classes_  

rows = []

for i in range(X_explain_small.shape[0]):

    true_label = int(y_explain_small[i])
    proba = best_model.predict_proba(X_explain_small[i].reshape(1, -1))[0]
    pred_class_idx = int(np.argmax(proba))         
    pred_label = int(class_ids[pred_class_idx])    

   
    if isinstance(shap_values, list):
      
        sv_i = shap_values[pred_class_idx][i]     
    else:
   
        sv_i = shap_values[i]
        if sv_i.ndim == 2:                         
            sv_i = sv_i[:, pred_class_idx]       


    abs_sv = np.abs(sv_i)
    top_idx = np.argsort(-abs_sv)[:10]          

    for rank, j in enumerate(top_idx, start=1):
        rows.append({
            "sample_local_index": i,
            "true_label": true_label,
            "true_label_name": activity_labels.get(true_label, "Unknown"),
            "pred_label": pred_label,
            "pred_label_name": activity_labels.get(pred_label, "Unknown"),
            "feature_rank": rank,
            "feature_index": int(j),
            "feature_name": feature_names[j],
            "feature_value": float(X_explain_small[i, j]),
            "shap_value": float(sv_i[j]),
            "abs_shap": float(abs_sv[j]),
        })

df_top10 = pd.DataFrame(rows)

print("Nombre de lignes dans le top10 :", df_top10.shape[0])
df_top10.head(30)


In [ ]:
import matplotlib.pyplot as plt


feat_global = (
    df_top10
    .groupby("feature_name")["abs_shap"]
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

top_n = 20  
top_feat_global = feat_global.head(top_n)

plt.figure(figsize=(8, 6))
plt.barh(top_feat_global["feature_name"], top_feat_global["abs_shap"])
plt.gca().invert_yaxis()
plt.xlabel("Importance moyenne |SHAP|")
plt.title(f"Top {top_n} features les plus importants (sur les échantillons expliqués)")
plt.tight_layout()
plt.show()


In [ ]:
def plot_top_features_for_activity(activity_name, top_n=15):
   
    subset = df_top10[df_top10["true_label_name"] == activity_name]
    if subset.empty:
        print(f"Aucun échantillon avec true_label_name = {activity_name} dans df_top10.")
        return

    feat_act = (
        subset
        .groupby("feature_name")["abs_shap"]
        .mean()
        .sort_values(ascending=False)
        .reset_index()
        .head(top_n)
    )

    plt.figure(figsize=(8, 6))
    plt.barh(feat_act["feature_name"], feat_act["abs_shap"])
    plt.gca().invert_yaxis()
    plt.xlabel("Importance moyenne |SHAP|")
    plt.title(f"Top {top_n} features pour l'activité : {activity_name}")
    plt.tight_layout()
    plt.show()


plot_top_features_for_activity("STANDING", top_n=15)


In [ ]:
def plot_feature_scatter(feature_name):
    subset = df_top10[df_top10["feature_name"] == feature_name]
    if subset.empty:
        print(f"Aucune ligne pour la feature {feature_name} dans df_top10.")
        return

    plt.figure(figsize=(6, 5))

    activities = subset["true_label_name"].astype(str).values
    unique_acts = sorted(subset["true_label_name"].unique())


    act_to_idx = {act: idx for idx, act in enumerate(unique_acts)}
    colors = [act_to_idx[a] for a in activities]

    scatter = plt.scatter(
        subset["feature_value"],
        subset["shap_value"],
        c=colors
    )
    plt.axhline(0, linestyle="--")
    plt.xlabel("Valeur de la feature")
    plt.ylabel("SHAP value (contribution)")
    plt.title(f"Impact de la feature : {feature_name}")


    handles = [
        plt.Line2D([0], [0], marker="o", linestyle="", label=act)
        for act in unique_acts
    ]
    plt.legend(handles=handles, title="true_label_name")
    plt.tight_layout()
    plt.show()


some_feature = feat_global.iloc[0]["feature_name"]
print("Exemple de feature :", some_feature)
plot_feature_scatter(some_feature)


In [ ]:

top_feats = (
    df_top10
    .groupby("feature_name")["abs_shap"]
    .mean()
    .sort_values(ascending=False)
    .head(15)              
    .index
)


pivot_df = (
    df_top10[df_top10["feature_name"].isin(top_feats)]
    .pivot_table(
        index="sample_local_index",
        columns="feature_name",
        values="abs_shap",
        aggfunc="max"  
    )
    .fillna(0)
)

plt.figure(figsize=(10, 6))
plt.imshow(pivot_df.values, aspect="auto")
plt.colorbar(label="abs_SHAP")
plt.xticks(range(len(pivot_df.columns)), pivot_df.columns, rotation=90)
plt.yticks(range(len(pivot_df.index)), pivot_df.index)
plt.xlabel("Feature")
plt.ylabel("sample_local_index")
plt.title("Heatmap des contribs SHAP (top features globales)")
plt.tight_layout()
plt.show()


In [ ]:
from lime import lime_tabular
from IPython.display import display, HTML

lime_explainer = lime_tabular.LimeTabularExplainer(
    X_scaled,
    feature_names=feature_names.tolist(),
    class_names=[activity_labels[i] for i in sorted(activity_labels.keys())],
    discretize_continuous=True,
    mode="classification"
)

i = 10  
exp = lime_explainer.explain_instance(
    X_scaled[i],
    best_model.predict_proba,
    num_features=10
)

html = exp.as_html()
display(HTML(html))


In [ ]:
import matplotlib.pyplot as plt

fig = exp.as_pyplot_figure()
plt.tight_layout()
plt.show()


In [ ]:
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from lime import lime_tabular

lime_explainer = lime_tabular.LimeTabularExplainer(
    X_scaled,
    feature_names=feature_names.tolist(),
    class_names=[activity_labels[i] for i in sorted(activity_labels.keys())],
    discretize_continuous=True,
    mode="classification"
)

i = 10
exp = lime_explainer.explain_instance(
    X_scaled[i],
    best_model.predict_proba,
    num_features=10
)

print("Vue graphique :")
fig = exp.as_pyplot_figure()
plt.tight_layout()
plt.show()

print("\n Vue tableau :")
display(HTML(exp.as_html()))


In [ ]:
from IPython.display import display, HTML
import matplotlib.pyplot as plt

def explain_with_lime(i, num_features=10):
    
    print(f" Sample index = {i}")
    true_label = int(y[i])
    proba = best_model.predict_proba(X_scaled[i].reshape(1, -1))[0]
    class_ids = best_model.classes_
    pred_class_idx = proba.argmax()
    pred_label = int(class_ids[pred_class_idx])

    print("Vraie classe   :", true_label, "-", activity_labels.get(true_label, "Unknown"))
    print("Classe prédite :", pred_label, "-", activity_labels.get(pred_label, "Unknown"))
    print("Probas         :", dict(zip(class_ids, proba.round(3))))

    exp = lime_explainer.explain_instance(
        X_scaled[i],
        best_model.predict_proba,
        num_features=num_features
    )


    fig = exp.as_pyplot_figure()
    plt.tight_layout()
    plt.show()


    display(HTML(exp.as_html()))


explain_with_lime(10)


In [ ]:
import numpy as np
import pandas as pd

rows_lime = []
samples_per_class = 3   

class_ids = best_model.classes_  

for cls in class_ids:

    idxs = np.where(y == cls)[0]

    if len(idxs) == 0:
        continue  


    k = min(samples_per_class, len(idxs))
    chosen = np.random.choice(idxs, size=k, replace=False)

    for i in chosen:
        exp = lime_explainer.explain_instance(
            X_scaled[i],
            best_model.predict_proba,
            num_features=10
        )

        true_label = int(cls)
        proba = best_model.predict_proba(X_scaled[i].reshape(1, -1))[0]
        pred_class_idx = np.argmax(proba)
        pred_label = int(class_ids[pred_class_idx])

        for rank, (cond, weight) in enumerate(exp.as_list(), start=1):
            rows_lime.append({
                "sample_index": i,
                "true_label": true_label,
                "true_label_name": activity_labels.get(true_label, "Unknown"),
                "pred_label": pred_label,
                "pred_label_name": activity_labels.get(pred_label, "Unknown"),
                "feature_rank": rank,
                "feature_condition": cond,
                "lime_weight": float(weight),
            })

df_lime = pd.DataFrame(rows_lime)
df_lime["true_label_name"].value_counts()


In [ ]:
feat_lime_global = (
    df_lime
    .groupby("feature_condition")["lime_weight"]
    .mean()
    .to_frame()
)

feat_lime_global["abs_weight"] = feat_lime_global["lime_weight"].abs()
feat_lime_global = feat_lime_global.sort_values("abs_weight", ascending=False)

feat_lime_global.head(20)


In [ ]:
import matplotlib.pyplot as plt

feat_lime_signed = (
    df_lime
    .groupby("feature_condition")["lime_weight"]
    .mean()
    .sort_values(ascending=True)    
    .to_frame()
    .reset_index()
)

top_n = 20

feat_lime_signed["abs_weight"] = feat_lime_signed["lime_weight"].abs()
feat_lime_signed = feat_lime_signed.sort_values("abs_weight", ascending=False).head(top_n)
feat_lime_signed = feat_lime_signed.sort_values("lime_weight")  

plt.figure(figsize=(9, 6))
plt.barh(
    feat_lime_signed["feature_condition"],
    feat_lime_signed["lime_weight"]
)
plt.axvline(0, linestyle="--")
plt.xlabel("Poids moyen LIME (positif / négatif)")
plt.title(f"Top {top_n} conditions LIME (avec signe)")
plt.tight_layout()
plt.show()


In [ ]:

top_conditions = (
    feat_lime_global
    .sort_values("abs_weight", ascending=False)
    .head(10)
    .index
)


pivot_lime = (
    df_lime[df_lime["feature_condition"].isin(top_conditions)]
    .groupby(["true_label_name", "feature_condition"])["lime_weight"]
    .mean()
    .abs()
    .reset_index()
    .pivot(index="true_label_name", columns="feature_condition", values="lime_weight")
    .fillna(0)
)

plt.figure(figsize=(10, 5))
plt.imshow(pivot_lime.values, aspect="auto")
plt.colorbar(label="|lime_weight| moyen")
plt.xticks(range(len(pivot_lime.columns)), pivot_lime.columns, rotation=90)
plt.yticks(range(len(pivot_lime.index)), pivot_lime.index)
plt.xlabel("Condition LIME")
plt.ylabel("true_label_name")
plt.title("Heatmap |LIME| par activité et condition")
plt.tight_layout()
plt.show()
